# 0.0.Entendimento Do Problema de Negócio

- Objetivo do Problema
    - 1.0.Previsão do preço final para cada casa.
- Por que?
    - Qual o Tipo de modelo Negocio que estamos lidando?
        - Estamos lidando com um tipo de négocio de compra e venda de casas.
           - Comprar casas por um preço menor e vender por um preço mais caro
    - Como a empresa ganha dinheiro?(A grosso modo)
        - A empresa ganha dinheiro comprando casas por um preço menor e vendendo por um preço maior.
            - Receita: Venda - Compra = Receita.
            
- Contexto:
    - Possivelmente o problema aqui e a automação da precificação dos preços das casas, pois a empresa está querendo comprar mais imóveis e não sabe o preço que aquele imóvel deveria estar, e por esse motivo acaba deixando passar boas oportunidades de compra ou até comprando imóveis que estão com o preço além do que deveria estar.
           
- Proposta de Solução
    - 1.0.Para descobrir o preço justo da casa eu vou fazer um modelo preditivo que dada as caracteristicas nos de o preço do imóvel.
    - 2.0.Como o modelo vai ajudar a empresa a ganhar mais dinheiro?
        - O modelo vai ajudar na hora de comprar um imóvel, como ele vai nos trazer a precificação que aquele imóvel deveria estar, o comprador vai saber exatamente se está comprando um imóvel que está com o preço abaixo ou acima do que deveria estar, tendo essa informação em mãoes ele vai poder negociar de uma melhor forma a compra de um imóvel, afetando assim diretamente a Receita da empresa, pois ele vai saber as melhores oportunidades de compra, consequentemente comprando mais barato e no futuro vendendo mais caro, assim aumentando a Receita da empresa.
    - 2.0.Produto Final(O que eu vou entregar no fim do projeto)
       - WebApp
         - Input: Recebe as caracteristicas da casa
         - Output: Retorna o preço da casa.

# 1.0.Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
#Show all columns
pd.set_option('display.max_columns', None)

## 1.1.Load Data

In [2]:
df_train = pd.read_csv('datasets/train.csv')

## 1.2.Help Functions

In [3]:
def avaliar_modelo(nome_modelo,y_test,previsao):
    mae = mean_squared_error(y_test,previsao)
    mape = mean_absolute_percentage_error(y_test,previsao)
    r2 = r2_score(y_test,previsao)
    RMSE = np.sqrt(mean_squared_error(y_test,previsao))
    return print('Modelo {}:\nR²: {:.2%}\nRMSE: {:.2f}\nMAE: {:.2f}\nMAPE: {:.2%}'.format(nome_modelo,r2,RMSE,mae,mape))

# 2.0.Data Descripition

In [4]:
df1 = df_train.copy()
df1.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## 2.1.Data Dimensions

In [5]:
print('Numbers of rows: {}'.format(df1.shape[0]))
print('Numbers of columns: {}'.format(df1.shape[1]))

Numbers of rows: 1460
Numbers of columns: 81


## 2.2.Data Types

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

## 2.3.Change Data Type

In [7]:
#None to change
df1.head(1)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500


## 2.4.NA Check

In [8]:
aux = df1.isna().sum().reset_index().rename(columns={0:'NAN_Values','index':'Attributes'})
aux[aux['NAN_Values'] > 0]

,Attributes,NAN_Values
3,LotFrontage,259
6,Alley,1369
25,MasVnrType,8
26,MasVnrArea,8
30,BsmtQual,37
31,BsmtCond,37
32,BsmtExposure,38
33,BsmtFinType1,37
35,BsmtFinType2,38
42,Electrical,1


# 3.0.Data Filtering

In [9]:
df2 = df1.copy()

## 3.1.Filter Rows

## 3.2.Filter Columns

In [10]:
#Drop columns with NA values
cols = ['LotFrontage','Alley','MasVnrType','MasVnrArea','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
 'Electrical','FireplaceQu','GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature', # Columns with NA Values
       'MoSold','YrSold','SaleType','SaleCondition',# Columns of business
        'Id'] #Ids
df2 = df2.drop(cols,axis=1)

In [11]:
df2.shape

(1460, 57)

# 4.0.Data Preparation

In [12]:
df3 = df2.copy()

In [13]:
#dummy variables
df3_dummy = pd.get_dummies(df3.drop('SalePrice',axis=1))

#join SalePrice with variables dummys
df3 = pd.concat([df3['SalePrice'],df3_dummy],axis=1)

In [14]:
#rows and columns
df3.shape

(1460, 199)

# 5.0.Machine Learning Model

In [15]:
df4 = df3.copy()

In [16]:
X = df4.drop('SalePrice',axis=1)
y = df4['SalePrice']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=32)

In [17]:
X_train.shape,X_test.shape, y_train.shape, y_test.shape

((1022, 198), (438, 198), (1022,), (438,))

## Linear model

In [18]:
ln = LinearRegression()
ln.fit(X_train,y_train)
previsao_ln = ln.predict(X_test)

## Random Forest

In [19]:
rf = RandomForestRegressor()
rf.fit(X_train,y_train)
previsao_rf = rf.predict(X_test)

# 6.0. Perfomance

In [20]:
avaliar_modelo('Linear Regression',y_test,previsao_ln)

Modelo Linear Regression:
R²: -24356030664.20%
RMSE: 1215665726.94
MAE: 1477843159661207040.00
MAPE: 99539.75%


In [21]:
avaliar_modelo('Random Forest Regression',y_test,previsao_rf)

Modelo Random Forest Regression:
R²: 82.48%
RMSE: 32606.39
MAE: 1063176681.69
MAPE: 11.81%


# Conclusion

Como podemos notar, o modelo de Linear Regression não perfomou bem e um indicativo que não estamos lidando com dados simples, mas sim complexos, pois a nossa Random Forest perfomou super bem, só que ainda falta outros passos para diminuimos o erro do modelo melhorando a precisão dele.

**Proximos passos:**

- Cross Validation
- Data Cleaning
- Feature Selection
- Create Baseline

Entre outros...